In [3]:
import spacy
nlp = spacy.load('en_core_web_lg')
import pandas as pd
import numpy as np
import gensim 
import logging
import glob, os
from gensim.models import word2vec, Phrases
from gensim.models.phrases import Phraser
from ast import literal_eval
import string

### 1. Use one of the larger SpaCy models containing vectors (I prefer “en_core_web_lg” but you can use any listed here under “Available models”: https://spacy.io/usage/models ).

### 1a. Choose 2 words/terms/tokens and provide the top 50 similarity scores for each.

In [23]:
def most_similar50(word):
    by_similarity = sorted(word.vocab, key=lambda w: word.similarity(w), reverse=True)
    return [str(w.orth_) + " : " + str(w.similarity(word)) for w in by_similarity[:50]]

In [22]:
similar1 = most_similar50(nlp.vocab[u'american'])

In [9]:
print(similar1)

['american : 1.0', 'American : 0.99999994', 'AMERICAN : 0.99999994', 'AMerican : 0.99999994', 'AMerica : 0.83003515', 'america : 0.83003515', 'America : 0.83003515', 'AMERICA : 0.83003515', 'British : 0.73649323', 'british : 0.73649323', 'BRITISH : 0.73649323', 'Canadian : 0.7126985', 'CANADIAN : 0.7126985', 'canadian : 0.7126985', 'EUropean : 0.71116835', 'EUROPEAN : 0.71116835', 'European : 0.71116835', 'european : 0.71116835', 'Americans : 0.7083187', 'americans : 0.7083187', 'AMERICANS : 0.7083187', 'AMericans : 0.7083187', 'African : 0.70086735', 'african : 0.70086735', 'AFRICAN : 0.70086735', 'Australian : 0.69194996', 'australian : 0.69194996', 'AUSTRALIAN : 0.69194996', 'UsA : 0.67251575', 'usa : 0.67251575', 'USa : 0.67251575', 'Usa : 0.67251575', 'USA : 0.67251575', 'MEXICAN : 0.6590071', 'Mexican : 0.6590071', 'mexican : 0.6590071', 'Indian : 0.64163625', 'indian : 0.64163625', 'INDIAN : 0.64163625', 'CHICAGO : 0.63874257', 'chicago : 0.63874257', 'CHicago : 0.63874257', 'Ch

In [8]:
similar2 = most_similar50(nlp.vocab[u'great'])

In [152]:
print(similar2)

['great : 1.0', 'GReat : 1.0', 'GREAT : 1.0', 'greAt : 1.0', 'Great : 1.0', 'good : 0.84167075', 'Good : 0.84167075', 'gOOD : 0.84167075', 'GOOD : 0.84167075', 'GOod : 0.84167075', 'FANTASTIC : 0.8124874', 'Fantastic : 0.8124874', 'FANtastic : 0.8124874', 'fantastic : 0.8124874', 'Wonderful : 0.805945', 'wonderful : 0.805945', 'WONDERFUL : 0.805945', 'amazing : 0.78232324', 'AMazing : 0.78232324', 'AMAZING : 0.78232324', 'Amazing : 0.78232324', 'amaZing : 0.78232324', 'EXCELLENT : 0.77073884', 'Excellent : 0.77073884', 'excellent : 0.77073884', 'awesome : 0.7549282', 'Awesome : 0.7549282', 'AWesome : 0.7549282', 'AWEsome : 0.7549282', 'AWESOME : 0.7549282', 'VERy : 0.7500455', 'VERY : 0.7500455', 'Very : 0.7500455', 'very : 0.7500455', 'VEry : 0.7500455', 'Terrific : 0.7480562', 'terrific : 0.7480562', 'TERRIFIC : 0.7480562', 'nICE : 0.73561585', 'NICE : 0.73561585', 'Nice : 0.73561585', 'nice : 0.73561585', 'NIce : 0.73561585', 'really : 0.7232506', 'reAlly : 0.7232506', 'REALLy : 0.7

### 1b. Add 3 words/terms/tokens to the 2 you chose for #1 and provide the 1-to-1 comparative similarity scores for all 5. (E.g. for “dog”, “cat”, “worm”, “tree” and “banana” provide dog-cat, dog-worm, dog-tree, dog-banana, cat-worm, cat-tree, cat-banana, worm-tree, worm-banana and tree-banana).

In [27]:
mytoken = nlp("american great beautiful strong make")
for n in range(len(mytoken)):
    for m in range(n+1,len(mytoken)):
        print(mytoken[n].text, mytoken[m].text, mytoken[n].similarity(mytoken[m]))

american great 0.27806786
american beautiful 0.22853203
american strong 0.20254456
american make 0.30104175
great beautiful 0.59625214
great strong 0.53746843
great make 0.5798683
beautiful strong 0.3536118
beautiful make 0.4160449
strong make 0.47789517


### 2. Using your corpus and the gensim package, train a NN (neural network) Word2Vec model, then provide the code (especially your parameters) and the following items.

In [80]:
# read cleaned data
df = pd.read_csv("../Project/df.csv")
df = df.drop(["Unnamed: 0"],axis = 1)
df.head()

,Speaker,FileName,RawText,Token,Cleaned_text,Sent_Token,Normalized_list
0,BUSH,../Project/Data/BUSH/BUSH-0.txt,thank you very much. i want to thank arizona ...,"['thank', 'much', 'want', 'thank', 'arizona', ...",thank much want thank arizona state well yes s...,"[' thank you very much.', 'i want to thank ari...","[['thank', 'much'], ['want', 'thank', 'arizona..."
1,BUSH,../Project/Data/BUSH/BUSH-1.txt,"gosh, i just do not think i ever said i am no...","['gosh', 'think', 'ever', 'said', 'worried', '...",gosh think ever said worried osama bin laden k...,"[' gosh, i just do not think i ever said i am ...","[['gosh', 'think', 'ever', 'say', 'worry', 'os..."
2,BUSH,../Project/Data/BUSH/BUSH-10.txt,i think it is important to promote a culture ...,"['think', 'important', 'promote', 'culture', '...",think important promote culture life think hos...,[' i think it is important to promote a cultur...,"[['think', 'important', 'promote', 'culture', ..."
3,BUSH,../Project/Data/BUSH/BUSH-11.txt,"gosh, i sure hope it is not the administratio...","['gosh', 'sure', 'hope', 'administration', 'lo...",gosh sure hope administration look systemic pr...,"[' gosh, i sure hope it is not the administrat...","[['gosh', 'sure', 'hope', 'administration'], [..."
4,BUSH,../Project/Data/BUSH/BUSH-12.txt,"i think it is important, since he talked abou...","['think', 'important', 'since', 'talked', 'med...",think important since talked medicare plan uni...,"[' i think it is important, since he talked ab...","[['think', 'important', 'since', 'talk', 'medi..."


In [81]:
# show speaker's name
df["Speaker"].unique()

array(['BUSH', 'CRUZ', 'HILLARY', 'KERRY', 'MCCAIN', 'OBAMA', 'ROMNEY',
       'SANDERS', 'TRUMP'], dtype=object)

In [82]:
# Group speakers' into two parties
Rep = ["TRUMP","CRUZ","ROMNEY","BUSH","MCCAIN"]
Dem = ["HILLARY","OBAMA","SANDERS","KERRY"]
Rep_df = df.loc[df['Speaker'].isin(Rep)][['Speaker','Normalized_list']]
Dem_df = df.loc[df['Speaker'].isin(Dem)][['Speaker','Normalized_list']]

In [84]:
Rep_df.head()

,Speaker,Normalized_list
0,BUSH,"[['thank', 'much'], ['want', 'thank', 'arizona..."
1,BUSH,"[['gosh', 'think', 'ever', 'say', 'worry', 'os..."
2,BUSH,"[['think', 'important', 'promote', 'culture', ..."
3,BUSH,"[['gosh', 'sure', 'hope', 'administration'], [..."
4,BUSH,"[['think', 'important', 'since', 'talk', 'medi..."


### 2a. Choose 5 words/terms/tokens and provide the 10 most similar words/terms/tokens from your corpus according to your model, plus the similiarity score for each match.

In [85]:
# clean data and covert string to list

def remove_punctuation(l):
    r = []
    not_wanted = ["‘","’",'”','“','–']
    for i in l:
        removed = [x for x in i if x not in not_wanted]
        r.append(removed)
    return r

Rep_df.Normalized_list = Rep_df.Normalized_list.apply(literal_eval)
Dem_df.Normalized_list = Dem_df.Normalized_list.apply(literal_eval)

Rep_sent = Rep_df.Normalized_list.sum()
Dem_sent = Dem_df.Normalized_list.sum()

Dem_sent = remove_punctuation(Dem_sent)
Rep_sent = remove_punctuation(Rep_sent)

In [111]:
Rep_trigram_sentences_project = []
Dem_trigram_sentences_project = []

# Set values for various parameters                                   [= "Tuning the Model"]
num_features = 150    # Word vector dimensionality                      
min_word_count = 1    # Minimum word count                        
num_workers = 20      # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

Dem_bigram = Phraser(Phrases(Dem_sent))
Dem_trigram = Phraser(Phrases(Dem_bigram[Dem_sent]))
Rep_bigram = Phraser(Phrases(Rep_sent))
Rep_trigram = Phraser(Phrases(Rep_bigram[Rep_sent]))

for sent in Rep_sent:
    bigrams_ = Rep_bigram[sent]
    trigrams_ = Rep_trigram[Rep_bigram[sent]]
    Rep_trigram_sentences_project.append(trigrams_)
    
for sent in Dem_sent:
    bigrams_ = Dem_bigram[sent]
    trigrams_ = Dem_trigram[Dem_bigram[sent]]
    Dem_trigram_sentences_project.append(trigrams_)

Rep_model = word2vec.Word2Vec(Rep_trigram_sentences_project, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, sg=1)           #sg=1 turns on skip-grams, otherwise CBOW

Dem_model = word2vec.Word2Vec(Dem_trigram_sentences_project, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, sg=1)

In [114]:
Dem_vocab = list(Dem_model.wv.vocab.keys())[:5]
Rep_vocab = list(Rep_model.wv.vocab.keys())[:5]

In [123]:
def return_similar_result(party,l,model):
    print('For',party,", the 10 most similar words/terms/tokens for the following words are:" )
    print("")
    for w in l:
        print(w)
        r = model.wv.most_similar (positive=w)
        print(r)
        print("")
    return r

In [124]:
Rep_r = return_similar_result("Republican party", Rep_vocab, Rep_model)

For Republican party , the 10 most similar words/terms/tokens for the following words are:

thank_much
[('leadership', 0.9982559680938721), ('market', 0.9982283711433411), ('constitutional', 0.9982229471206665), ('longterm', 0.9982197284698486), ('thought', 0.9982196092605591), ('risk', 0.9982132911682129), ('behind', 0.998207688331604), ('around', 0.9982069730758667), ('sign', 0.9981980919837952), ('wife', 0.9981955289840698)]

want_thank
[('guy', 0.9997182488441467), ('medicare', 0.9997088313102722), ('head', 0.999699592590332), ('technology', 0.9996991753578186), ('pretty', 0.9996957182884216), ('citizen', 0.9996956586837769), ('succeed', 0.9996895790100098), ('account', 0.9996885061264038), ('iraq', 0.9996881484985352), ('united_state_america', 0.999687910079956)]

arizona
[('poverty', 0.9991181492805481), ('founding', 0.9991021156311035), ('standing', 0.9990628957748413), ('abortion', 0.9990487098693848), ('daughter', 0.9990425109863281), ('remain', 0.9990195035934448), ('call', 0

In [125]:
Dem_r = return_similar_result("Democratic party", Dem_vocab, Dem_model)

For Democratic party , the 10 most similar words/terms/tokens for the following words are:

know
[('away', 0.9997724890708923), ('government', 0.9997686743736267), ('far', 0.9997611045837402), ('company', 0.9997572302818298), ('oil', 0.999756932258606), ('fact', 0.9997559189796448), ('since', 0.9997555613517761), ('would', 0.9997535943984985), ('people', 0.9997533559799194), ('order', 0.9997531175613403)]

take
[('process', 0.9997642040252686), ('never', 0.9997590780258179), ('go', 0.9997581839561462), ('war', 0.999754786491394), ('support', 0.9997515082359314), ('country', 0.9997515082359314), ('care', 0.9997506737709045), ('leadership', 0.999748945236206), ('like', 0.9997448325157166), ('run', 0.9997419118881226)]

get
[('go', 0.9997610449790955), ('mean', 0.9997559189796448), ('provide', 0.9997465014457703), ('special', 0.9997453093528748), ('see', 0.9997438192367554), ('political', 0.9997432827949524), ('billion', 0.9997409582138062), ('trade_agreement', 0.9997403621673584), ('effo

## 2b. Provide the relative cosine similarity for two word pairs, based on your model.

In [1]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [141]:
print('For',"Republican party",", the relative cosine similarity for two word pairs are:" )
print("")
for n in Rep_vocab:
    for m in Rep_r:
        print(n,m[0],Rep_model.wv.relative_cosine_similarity(n, m[0]))

For Republican party , the relative cosine similarity for two word pairs are:

thank_much create 0.09998488592118782
thank_much technology 0.09998999719309964
thank_much raise 0.09997955968924231
thank_much joe_plumber 0.09997454395512323
thank_much happen 0.09997299743710317
thank_much source 0.09998459930780959
thank_much price 0.09998461125003368
thank_much whole 0.0999909645132512
thank_much senator 0.09997740411779352
thank_much politics 0.09999206319786776
want_thank create 0.09999269741142314
want_thank technology 0.10000020390964184
want_thank raise 0.09998937046463519
want_thank joe_plumber 0.09998859536950538
want_thank happen 0.09998630585773732
want_thank source 0.09999666828339586
want_thank price 0.09999713334047375
want_thank whole 0.09999642383031646
want_thank senator 0.09999367522374075
want_thank politics 0.09999843907765396
arizona create 0.09998532143331518
arizona technology 0.09997625882192865
arizona raise 0.09997587102091475
arizona joe_plumber 0.09998023229077

In [142]:
print('For',"Democratic party",", the relative cosine similarity for two word pairs are:" )
print("")
for n in Dem_vocab:
    for m in Dem_r:
        print(n,m[0],Dem_model.wv.relative_cosine_similarity(n, m[0]))

For Democratic party , the relative cosine similarity for two word pairs are:

know three 0.09999691590783621
know something 0.09999631971758612
know try 0.0999944894135183
know strong 0.09999704110778873
know oil 0.09999981339245172
know bring 0.09999823945019144
know fight 0.09999903238322408
know agree 0.09999425093741825
know member 0.09998860501574977
know friend 0.0999950617561584
take three 0.09998683782576717
take something 0.09999498183537085
take try 0.0999882985009889
take strong 0.09999064750522306
take oil 0.09999464796674874
take bring 0.09999615633748793
take fight 0.09999709832395746
take agree 0.09999162526333068
take member 0.09999589997408166
take friend 0.09999494010179308
get three 0.09999200975198978
get something 0.09999654682014064
get try 0.09999415010345648
get strong 0.09999887795502498
get oil 0.09999712513237537
get bring 0.09999710724642996
get fight 0.09999180704460854
get agree 0.09999367910689418
get member 0.09998824177949131
get friend 0.0999933810078

### 2c.Provide an example of an analogy from your model. (Don't worry if it seems to make no sense.)

In [146]:
Rep_pool = set(Rep_model.wv.vocab.keys())
Dem_pool = set(Dem_model.wv.vocab.keys())
print(Rep_pool.intersection(Dem_pool))

{'also', 'israel', 'record', 'mass', 'discuss', 'single', 'fill', 'add', 'fund', 'background', 'chris', 'v', 'bernie', 'average', 'arm', 'doctor', 'relation', 'mean', 'iranian', 'journey', 'campaigning', 'establish', 'everyone', 'stand', 'interest', 'officer', 'lead', 'dishonest', 'page', 'body', 'tuesday', 'quickly', 'nevada', 'movement', 'easier', 'majority', 'patriot', 'career', 'applaud', 'target', 'think', 'longterm', 'behind', 'hand', 'zero', 'month', 'three', 'government', 'reserve', 'primary', 'social_security', '8', 'third', 'take', 'burn', 'doubt', 'amazing', 'unbelievable', 'discrimination', 'goal', 'mayor', 'even', 'want_thank', 'insurance', 'apology', 'neither', '911', 'heal', 'lie', 'additional', 'wrong', 'oppose', 'adult', '2000', 'car', 'vote', 'lack', 'child', 'credit', 'gut', 'socialism', 'south', 'workforce', 'senate', 'payment', 'freeze', 'certain', 'trust', 'fifth', 'pose', 'assure', 'global', 'integrity', 'europe', 'vibrant', 'half', 'aside', 'thank_much', 'show',

In [147]:
Rep_model.wv.most_similar(positive=["republican",'great'],negative=['democratic'])

[('convention', 0.9980177283287048),
 ('deal', 0.9979611039161682),
 ('last', 0.9979349970817566),
 ('send', 0.9979149103164673),
 ('democrat', 0.9979007840156555),
 ('message', 0.9978934526443481),
 ('opportunity', 0.9978873133659363),
 ('individual', 0.9978849291801453),
 ('hope', 0.9978833198547363),
 ('iran', 0.9978702068328857)]

In [148]:
Dem_model.wv.most_similar(positive=['democratic','great'],negative=['republican'])

[('young', 0.9993492364883423),
 ('health_care', 0.999322772026062),
 ('system', 0.9993202686309814),
 ('interest', 0.9993100166320801),
 ('away', 0.9992996454238892),
 ('kid', 0.9992901086807251),
 ('cut', 0.9992859363555908),
 ('little', 0.9992716312408447),
 ('hour', 0.9992683529853821),
 ('play', 0.9992647171020508)]

### 2d. Provide the top 30 predicted output words for a series of words/tokens based on your model.

In [150]:
Rep_model.predict_output_word(["make","america","great"], topn=30)

[('kansas', 0.00068603543),
 ('trust', 0.0006403404),
 ('energy', 0.0006362638),
 ('party', 0.0006152094),
 ('existential', 0.000599299),
 ('socioeconomic', 0.00059543166),
 ('spring', 0.0005950942),
 ('fight', 0.0005873365),
 ('warrior', 0.00057709054),
 ('rare', 0.00056402874),
 ('freedom', 0.000556502),
 ('jeopardy', 0.0005560527),
 ('mom', 0.00055372505),
 ('gender', 0.0005479103),
 ('timeline', 0.00053967576),
 ('lowcost', 0.0005386202),
 ('protest', 0.0005295031),
 ('double', 0.0005220721),
 ('holiday', 0.00051562046),
 ('american', 0.0005123059),
 ('next', 0.0005060141),
 ('regulation', 0.0005037683),
 ('unbundle', 0.0005036505),
 ('onto', 0.0004889844),
 ('pool', 0.00048536542),
 ('grandkids', 0.00048443698),
 ('reaganlike', 0.000481958),
 ('1012', 0.00048182756),
 ('hardest', 0.00047990138),
 ('support_candidate', 0.00047845615)]

In [151]:
Dem_model.predict_output_word(["make","america","great"], topn=30)

[('gap', 0.0008641921),
 ('math', 0.00080421846),
 ('requirement', 0.0007871303),
 ('rigorous', 0.0007852309),
 ('unfunded', 0.00078021444),
 ('motor', 0.00077243405),
 ('68', 0.00077205116),
 ('42000', 0.0007463777),
 ('mandate', 0.0007407947),
 ('tired', 0.0007349613),
 ('original', 0.00073265715),
 ('ranking', 0.0007237611),
 ('perpetrate', 0.0007193926),
 ('turbine', 0.0007146641),
 ('prochoice', 0.0007146525),
 ('unaffordable', 0.0007135297),
 ('lifting', 0.0007107125),
 ('suppose', 0.00070941605),
 ('deployment', 0.0007052809),
 ('organize', 0.00070410135),
 ('20000', 0.0007040752),
 ('acre', 0.0006909125),
 ('generally', 0.00068605633),
 ('northwestern', 0.0006832074),
 ('swiftly', 0.00068274734),
 ('highquality', 0.0006789532),
 ('unintended', 0.0006775786),
 ('reprioritize', 0.0006767195),
 ('mccainkerry', 0.0006764001),
 ('perfectly', 0.0006738936)]

### 3.In a short paragraph (roughly 3-5 sentences), provide your thoughts on word vectors in the SpaCy model, your thoughts on building the model in gensim and what the model identifies as “similar” terms, and your thoughts on word vectors in general.

It's not meaningful for providing words with uppercase letter in the same words in the SpaCy model.The model in gensim may be more helpful in analyzing text data as it is built with my data and the results are closely related with the context. I would select only nouns for future analysis which might better reflect topics.